SOP0127 - Restore Keys For Encryption At Rest
=============================================

Description
-----------

Use this notebook to connect to the `controller` database and restore
keys for encryption at rest.

Steps
-----

### Parameters

Set the `file_to_restore_from`, where we can restore your encryption
keys from. Please make sure it has json file extension.

Set the `certificate_protection_password`. This is the password which
was used to encrypt your certificate.

In [ ]:
import os.path
import json

file_to_restore_from = "" # Change to your path there.
certificate_protection_password = "your_password"

print(f"Key(s) will be restored from: '{file_to_restore_from}'. Please make sure you have permission to access this path.")

if os.path.isfile(file_to_restore_from) == False:
    raise SystemExit(f"{file_to_restore_from} does not point to a valid file.")

with open(file_to_restore_from) as json_file:
    try:
        backup_json_data = json.load(json_file)
    except ValueError:
        raise SystemExit(f"{file_to_restore_from} does not have JSON content")


### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream

    if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
        config.load_incluster_config()
    else:
        try:
            config.load_kube_config()
        except:
            display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
            raise
    api = client.CoreV1Api()

    print('Kubernetes client instantiated')
except ImportError:
    display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
    raise

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Python function queries `controller` database and return results.

### Create helper function to run `sqlcmd` against the controller database

In [ ]:
import pandas
from io import StringIO
pandas.set_option('display.max_colwidth', -1)
name = 'controldb-0'
container = 'mssql-server'

def run_sqlcmd(query, show_count):
    
    no_count_string=""
    no_count_suffix=""
    if(show_count == False):
        no_count_string="SET NOCOUNT ON; "
        no_count_suffix = f""" | sed 2d"""

    command=f"""export SQLCMDPASSWORD=$(cat /var/run/secrets/credentials/mssql-sa-password/password);
    /opt/mssql-tools/bin/sqlcmd -b -S . -U sa -Q "{no_count_string}
    {query}" -d controller  -s"^" -W {no_count_suffix}
    """
    output=stream(api.connect_get_namespaced_pod_exec, name, namespace, command=['/bin/sh', '-c', command], container=container, stderr=True, stdout=True)
    return str(output)
print("Function 'run_sqlcmd' defined")

### Python function to execute kubernetes command.

In [ ]:
from io import StringIO
name = 'controldb-0'
container = 'mssql-server'

def execute_k8scommand(command):
    output=stream(api.connect_get_namespaced_pod_exec, name, namespace, command=['/bin/sh', '-c', command], container=container, stderr=True, stdout=True)
    return str(output)
print("Function 'execute_k8scommand' defined")

### Define function to check presence of keys.

In [ ]:
import json

def can_import_keys(backup_json_file_path):
  key_name_list = []

  key_name_list = [f"""'{backup_entry["id"]}'""" for backup_entry in backup_json_data]

  keys_list_string = ','.join(key_name_list)
  print(f"""Key names found in backup file {keys_list_string}""")
  sql_check_existing_keys = f"""select count(*) from Credentials where account_name in ({keys_list_string}) and type in ('2','3')"""

  count_existing_keys = run_sqlcmd(sql_check_existing_keys, False)

  can_import = False;

  if(0 == int(count_existing_keys)):
      can_import = True;
  else:
      print(f"""One or more keys from the list {keys_list_string} already exist in the Control plane. Please delete all the keys before importing them.""");
  
  return can_import
print("Function 'can_import_keys' defined")

### Generate in memory Base64URL encoded PFX without password protection.

In [ ]:
import base64

def generate_unprotected_pfx(base64_encoded_pfx, password):
    command=f"""
        generateUnprotectedPfx() {{
            {{ echo $1 | base64 -d | openssl pkcs12 -nocerts -nodes -passin pass:$(echo '{password}') 2>/dev/null ; \
            echo $1 | base64 -d | openssl pkcs12 -nokeys -passin pass:$(echo '{password}') 2>/dev/null ; }} \
            | openssl pkcs12 -export -password pass:$(echo '') 2>/dev/null | base64 -w 0
        }}

        generateUnprotectedPfx {base64_encoded_pfx}
    """

    command_output=execute_k8scommand(command)

    # Convert the base64 encoded string to a base64 url encoded string
    decoded_certificate = base64.b64decode(command_output)
    url_encoded_certificate = base64.urlsafe_b64encode(decoded_certificate); 
    return url_encoded_certificate.decode('utf-8')

print("Function 'generate_unprotected_pfx' defined.")

### Restore encryption keys.

Restore the keys from key backup json file, into the Big Data Cluster
control plane.

In [ ]:
import json
import base64

if(can_import_keys(file_to_restore_from) == False):
    from IPython.display import Markdown
    print("The keys cannot be imported since one or more keys exist. Delete the keys and re-run this cell before proceeding")
    display(Markdown(f'HINT: Use [SOP0124 - List Keys For Encryption At Rest.](../tde/sop124-list-keys-encryption-at-rest.ipynb) to resolve this issue.'))
    display(Markdown(f'HINT: Use [SOP0125 - Delete Key For Encryption At Rest](../tde/sop125-delete-keys-encryption-at-rest.ipynb) to resolve this issue.'))
else:
    key_password = base64.b64decode(str(api.read_namespaced_secret("controller-db-rw-secret", namespace).data['encryptionPassword'])).decode('utf-8')
    tsql_values_template = """('{account_name}', {type}, EncryptByKey(Key_GUID('ControllerDbSymmetricKey'), N'{secret_credential}'), N'{application_metadata}')"""
    value_list = []
    
    for backup_entry in backup_json_data:
        secret_value = ""
        entry_type = backup_entry["type"]
        if(entry_type == 2):
            secret_value = backup_entry["value"]
        elif(entry_type == 3):
            encoded_backup_certificate = backup_entry["value"]
            base64url_encoded_certificate = generate_unprotected_pfx(encoded_backup_certificate, certificate_protection_password)
            secret_value = base64url_encoded_certificate
        else:
            raise SystemExit(f"""Invalid type found {entry_type}""")
        
        value_list.append(tsql_values_template.format(account_name = backup_entry["id"], type = backup_entry["type"], secret_credential = secret_value, application_metadata = backup_entry["tags"].replace('"', '\\"')))
        print(f"""Encryption Key {backup_entry["id"]} processed.""")
    
    tsql_value_string = ','.join(value_list)
    tsql_insert_statement = f"""
    OPEN SYMMETRIC KEY ControllerDbSymmetricKey DECRYPTION BY PASSWORD = '{key_password}'
    INSERT INTO Credentials VALUES {tsql_value_string}
    """

    run_output = run_sqlcmd(tsql_insert_statement, True)
    print(run_output)


In [ ]:
print('Notebook execution complete.')